In [136]:
import cv2
import numpy as np
np.set_printoptions(threshold=np.inf)


In [137]:
def show(image):
    cv2.imshow("Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [138]:
image_path = "D:\Assignment 1\Q1\\test\\9.png"
image= cv2.imread(image_path,cv2.IMREAD_UNCHANGED)
if image is None:
    print("Error: Unable to load image.")
show(image)


In [139]:
image.shape

(475, 482, 4)

In [140]:
if image.shape[-1] == 4:
    image[image[:, :, 3] < 240] = [0,0,0,255]  
    
gray_image = cv2.cvtColor(image, cv2.COLOR_BGRA2GRAY)


In [141]:

# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Apply Gaussian blur to reduce noise and improve contour detection
# blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# # Threshold the image
# _, thresholded = cv2.threshold(blurred, 10, 255, cv2.THRESH_BINARY)

# # Find contours
# contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# quadrilaterals = []

# for contour in contours:
#     if len(contour) == 4:
#         quadrilaterals.append(contour)

# for quadrilateral in quadrilaterals:
#     cv2.drawContours(thresholded, [quadrilateral], -1, (0, 255, 0), 2)

# show(thresholded)
# contours

In [142]:
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# # blurred = cv2.GaussianBlur(gray, (5, 5), 0)
# # show(blurred)
# # edges = cv2.Canny(blurred, 100,200)
# # contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # quadrilateral_corners = []
# # for contour in contours:
# #     epsilon = 0.04 * cv2.arcLength(contour, True)
# #     approx = cv2.approxPolyDP(contour, epsilon, True)
# #     if len(approx) == 4:
# #         quadrilateral_corners.append(approx)

# # for corners in quadrilateral_corners:
# #     cv2.drawContours(blurred, [corners], 0, (0, 255, 0), 2)

# # show(blurred)
# blurred = cv2.GaussianBlur(gray, (5, 5), 0)
# gray = np.float32(gray)
# ret, thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
# contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # show(thresh)
# # corners = cv2.cornerHarris(thresh, blockSize=2, ksize=3, k=0.04)
# # # corners
# # threshold = 0.01 * corners.max()
# # corner_image = thresh.copy()
# # show(corner_image)

# # for i in range(corners.shape[0]):
# #     for j in range(corners.shape[1]):
# #         if corners[i, j] > threshold:
#             cv2.circle(corner_image, (j, i), 5, (0, 0, 255), 2)
# show(corner_image)

In [143]:
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = gray_image
cv2.imwrite("gray.jpg", gray_image)
ret, thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)

contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for contour in contours:
    points = cv2.approxPolyDP(contour, 0.04 * cv2.arcLength(contour, True), True)

p = []

for point in points:
    p.append(point[0])

print(p)

[array([370,   2], dtype=int32), array([148,   9], dtype=int32), array([  2, 472], dtype=int32), array([479, 472], dtype=int32)]


In [144]:

p = sorted(p, key=lambda x: x[0])
temp = p[1]
if(p[0][1] > p[1][1]):
    p[1] = p[0]
    p[0] = temp
temp = p[2]
if(p[2][1] < p[3][1]):
    p[2] = p[3]
    p[3] = temp

In [145]:
pts1 = np.float32(p)
pts2 = np.float32([[0,0],[0,600],[600,600],[600,0]])
print(pts1,pts2)

[[148.   9.]
 [  2. 472.]
 [479. 472.]
 [370.   2.]] [[  0.   0.]
 [  0. 600.]
 [600. 600.]
 [600.   0.]]


In [208]:
M = cv2.getPerspectiveTransform(pts1,pts2)
dst = cv2.warpPerspective(image,M,(600,600))

In [209]:
# kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
# sharpened_image = cv2.filter2D(dst, -1, kernel)

In [210]:
cv2.imwrite("test_image.jpg", dst)
cv2.imshow("Image", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [211]:
dst = dst[:,:,0:3]

In [212]:
show(dst)

In [213]:


# Define color thresholds
green_color = np.array([0, 128, 1], dtype=np.uint8)
orange_color = np.array([255, 153, 52], dtype=np.uint8)
tolerance = [10,10,10]  # Adjust this value to control the color matching tolerance

# Create masks for pixels that are close to the perfect colors
green_mask = cv2.inRange(dst, green_color - tolerance, green_color + tolerance)
orange_mask = cv2.inRange(dst, orange_color - tolerance, orange_color + tolerance)

# Create a mask for non-matching pixels
non_matching_mask = ~(green_mask | orange_mask)

# Set non-matching pixels to white
dst[non_matching_mask > 0] = [255, 255, 255]
show(dst)
cv2.imwrite('processed_image.jpg', dst)


True

In [214]:
# import cv2
# import numpy as np

# # Load the blurry image
# blurry_image = dst

# # Apply denoising to reduce noise
# denoised_image = cv2.fastNlMeansDenoisingColored(blurry_image, None, 10, 10, 7, 21)

# # Apply sharpening to enhance details
# kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
# sharpened_image = cv2.filter2D(denoised_image, -1, kernel)


# # Convert to HSV color space
# hsv_image = cv2.cvtColor(sharpened_image, cv2.COLOR_BGR2HSV)

# # Define the lower and upper bounds of the color in HSV
# lower_bound = np.array([0, 0, 0])
# upper_bound = np.array([255, 255, 255])

# # Create a mask for the colors within the specified range
# mask = cv2.inRange(hsv_image, lower_bound, upper_bound)

# # Apply the mask to the original image to extract the flag's colors
# extracted_colors = cv2.bitwise_and(sharpened_image, sharpened_image, mask=mask)

# # Resize the extracted colors to a higher resolution
# enlarged_colors = cv2.resize(extracted_colors, (600,600), interpolation=cv2.INTER_LINEAR)
# show(enlarged_colors)
# cv2.imwrite('enhanced_flag.jpg', enlarged_colors)


